# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
admission_predict = pd.read_csv(r'C:\Users\Daniel Carvalho\Desktop\DataAnalysis\WEEK2\DAY3\labs\lab-advanced-pandas\your-code\data\Admission_Predict.csv')
admission_predict.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.


In [3]:
admission_predict.columns = admission_predict.columns.str.strip()

Let's evaluate the dataset by looking at the `head` function.

In [4]:
admission_predict.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
admission_predict.notnull() #need to check this


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
380,True,True,True,True,True,True,True,True,True
381,True,True,True,True,True,True,True,True,True
382,True,True,True,True,True,True,True,True,True
383,True,True,True,True,True,True,True,True,True


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
pd.DataFrame.set_index(admission_predict,'Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
print(admission_predict.set_index(['GRE Score','CGPA']).index.is_unique)

True


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [8]:
admission_predict['GRE SCORE_CGPA'] = admission_predict.groupby(['GRE Score','CGPA'])
admission_predict.drop(columns=['GRE Score', 'CGPA'])
pd.DataFrame.set_index(admission_predict,'GRE SCORE_CGPA')

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
GRE SCORE_CGPA,,,,,,,,,
"((290, 7.46), [Serial No.])",1,337,118,4,4.5,4.5,9.65,1,0.92
"((290, 7.56), [Serial No.])",2,316,104,3,3.0,3.5,8.00,1,0.72
"((293, 7.8), [Serial No.])",3,322,110,3,3.5,2.5,8.67,1,0.80
"((294, 7.36), [Serial No.])",4,314,103,2,2.0,3.0,8.21,0,0.65
"((294, 7.64), [Serial No.])",5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
"((340, 9.6), [Serial No.])",381,324,110,3,3.5,3.5,9.04,1,0.82
"((340, 9.66), [Serial No.])",382,325,107,3,3.0,3.5,9.11,1,0.84
"((340, 9.74), [Serial No.])",383,330,116,4,5.0,4.5,9.45,1,0.91


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [9]:
admission_predict.reset_index()

,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE SCORE_CGPA
0,0,1,337,118,4,4.5,4.5,9.65,1,0.92,"((290, 7.46), [Serial No.])"
1,1,2,316,104,3,3.0,3.5,8.00,1,0.72,"((290, 7.56), [Serial No.])"
2,2,3,322,110,3,3.5,2.5,8.67,1,0.80,"((293, 7.8), [Serial No.])"
3,3,4,314,103,2,2.0,3.0,8.21,0,0.65,"((294, 7.36), [Serial No.])"
4,4,5,330,115,5,4.5,3.0,9.34,1,0.90,"((294, 7.64), [Serial No.])"
...,...,...,...,...,...,...,...,...,...,...,...
380,380,381,324,110,3,3.5,3.5,9.04,1,0.82,"((340, 9.6), [Serial No.])"
381,381,382,325,107,3,3.0,3.5,9.11,1,0.84,"((340, 9.66), [Serial No.])"
382,382,383,330,116,4,5.0,4.5,9.45,1,0.91,"((340, 9.74), [Serial No.])"
383,383,384,312,103,3,3.5,4.0,8.78,0,0.67,"((340, 9.91), [Serial No.])"


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [10]:
admission_predict[(admission_predict['CGPA'] > 9) & (admission_predict['Research'] == 1)]

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE SCORE_CGPA
0,1,337,118,4,4.5,4.5,9.65,1,0.92,"((290, 7.46), [Serial No.])"
4,5,330,115,5,4.5,3.0,9.34,1,0.90,"((294, 7.64), [Serial No.])"
10,11,328,112,4,4.0,4.5,9.10,1,0.78,"((296, 7.54), [Serial No.])"
19,20,328,116,5,5.0,5.0,9.50,1,0.94,"((298, 7.5), [Serial No.])"
20,21,334,119,5,5.0,4.5,9.70,1,0.95,"((298, 7.6), [Serial No.])"
...,...,...,...,...,...,...,...,...,...,...
379,380,329,111,4,4.5,4.0,9.23,1,0.89,"((340, 9.5), [Serial No.])"
380,381,324,110,3,3.5,3.5,9.04,1,0.82,"((340, 9.6), [Serial No.])"
381,382,325,107,3,3.0,3.5,9.11,1,0.84,"((340, 9.66), [Serial No.])"
382,383,330,116,4,5.0,4.5,9.45,1,0.91,"((340, 9.74), [Serial No.])"


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [11]:
adm = admission_predict[(admission_predict['CGPA'] > 9) & (admission_predict['SOP'] < 3.5)]

adm.mean()


C:\Users\Daniel Carvalho\AppData\Local\Temp\ipykernel_9888\331710087.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  adm.mean()


Serial No.           166.600
GRE Score            328.000
TOEFL Score          112.800
University Rating      3.400
SOP                    3.000
LOR                    3.400
CGPA                   9.130
Research               0.800
Chance of Admit        0.802
dtype: float64

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [12]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    result = admission_predict['col']-admission_predict['col'].mean()/admission_predict['col'].std()
    return result

In [13]:
admission_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE SCORE_CGPA
0,1,337,118,4,4.5,4.5,9.65,1,0.92,"((290, 7.46), [Serial No.])"
1,2,316,104,3,3.0,3.5,8.00,1,0.72,"((290, 7.56), [Serial No.])"
2,3,322,110,3,3.5,2.5,8.67,1,0.80,"((293, 7.8), [Serial No.])"
3,4,314,103,2,2.0,3.0,8.21,0,0.65,"((294, 7.36), [Serial No.])"
4,5,330,115,5,4.5,3.0,9.34,1,0.90,"((294, 7.64), [Serial No.])"
...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,"((340, 9.6), [Serial No.])"
381,382,325,107,3,3.0,3.5,9.11,1,0.84,"((340, 9.66), [Serial No.])"
382,383,330,116,4,5.0,4.5,9.45,1,0.91,"((340, 9.74), [Serial No.])"
383,384,312,103,3,3.5,4.0,8.78,0,0.67,"((340, 9.91), [Serial No.])"


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [14]:
admission_predict['GRE_std'] = (admission_predict['GRE Score']-admission_predict['GRE Score'].mean())/admission_predict['GRE Score'].std()
admission_predict['CGPA_std']=(admission_predict['CGPA']-admission_predict['CGPA'].mean())/admission_predict['CGPA'].std()
admission_predict['LOR_std']=(admission_predict['LOR']-admission_predict['LOR'].mean())/admission_predict['LOR'].std()

We will generate the decision choice at random using the code below. Please run the cell.

In [15]:
# Libraries
from random import choices

In [16]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']
decision_choice = choices(std_columns, k=admission_predict.shape[0])
admission_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE SCORE_CGPA,GRE_std,CGPA_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,"((290, 7.46), [Serial No.])",1.753381,1.747900,1.191647
1,2,316,104,3,3.0,3.5,8.00,1,0.72,"((290, 7.56), [Serial No.])",-0.063367,-0.991211,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.80,"((293, 7.8), [Serial No.])",0.455704,0.121034,-1.038166
3,4,314,103,2,2.0,3.0,8.21,0,0.65,"((294, 7.36), [Serial No.])",-0.236391,-0.642597,-0.480713
4,5,330,115,5,4.5,3.0,9.34,1,0.90,"((294, 7.64), [Serial No.])",1.147798,1.233279,-0.480713
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,"((340, 9.6), [Serial No.])",0.628727,0.735259,0.076740
381,382,325,107,3,3.0,3.5,9.11,1,0.84,"((340, 9.66), [Serial No.])",0.715239,0.851463,0.076740
382,383,330,116,4,5.0,4.5,9.45,1,0.91,"((340, 9.74), [Serial No.])",1.147798,1.415886,1.191647
383,384,312,103,3,3.5,4.0,8.78,0,0.67,"((340, 9.91), [Serial No.])",-0.409414,0.303641,0.634193


Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [24]:
admission_predict['decision_column'] = admission_predict.lookup(admission_predict.index,decision_choice)
admission_predict

C:\Users\Daniel Carvalho\AppData\Local\Temp\ipykernel_9888\3587161374.py:1: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  admission_predict['decision_column'] = admission_predict.lookup(admission_predict.index,decision_choice)


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE SCORE_CGPA,GRE_std,CGPA_std,LOR_std,decision_column
0,1,337,118,4,4.5,4.5,9.65,1,0.92,"((290, 7.46), [Serial No.])",1.753381,1.747900,1.191647,1.747900
1,2,316,104,3,3.0,3.5,8.00,1,0.72,"((290, 7.56), [Serial No.])",-0.063367,-0.991211,0.076740,0.076740
2,3,322,110,3,3.5,2.5,8.67,1,0.80,"((293, 7.8), [Serial No.])",0.455704,0.121034,-1.038166,0.455704
3,4,314,103,2,2.0,3.0,8.21,0,0.65,"((294, 7.36), [Serial No.])",-0.236391,-0.642597,-0.480713,-0.642597
4,5,330,115,5,4.5,3.0,9.34,1,0.90,"((294, 7.64), [Serial No.])",1.147798,1.233279,-0.480713,1.147798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,"((340, 9.6), [Serial No.])",0.628727,0.735259,0.076740,0.076740
381,382,325,107,3,3.0,3.5,9.11,1,0.84,"((340, 9.66), [Serial No.])",0.715239,0.851463,0.076740,0.715239
382,383,330,116,4,5.0,4.5,9.45,1,0.91,"((340, 9.74), [Serial No.])",1.147798,1.415886,1.191647,1.191647
383,384,312,103,3,3.5,4.0,8.78,0,0.67,"((340, 9.91), [Serial No.])",-0.409414,0.303641,0.634193,0.303641


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [38]:


def decision_func(row):
    if row['decision_column'] > 0.8:
        val = 1
    else:
        val= 0
    return val

admission_predict['decision'] = admission_predict.apply(decision_func, axis=1)
admission_predict

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE SCORE_CGPA,GRE_std,CGPA_std,LOR_std,decision_column,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,"((290, 7.46), [Serial No.])",1.753381,1.747900,1.191647,1.747900,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,"((290, 7.56), [Serial No.])",-0.063367,-0.991211,0.076740,0.076740,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,"((293, 7.8), [Serial No.])",0.455704,0.121034,-1.038166,0.455704,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,"((294, 7.36), [Serial No.])",-0.236391,-0.642597,-0.480713,-0.642597,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,"((294, 7.64), [Serial No.])",1.147798,1.233279,-0.480713,1.147798,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,"((340, 9.6), [Serial No.])",0.628727,0.735259,0.076740,0.076740,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,"((340, 9.66), [Serial No.])",0.715239,0.851463,0.076740,0.715239,0
382,383,330,116,4,5.0,4.5,9.45,1,0.91,"((340, 9.74), [Serial No.])",1.147798,1.415886,1.191647,1.191647,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,"((340, 9.91), [Serial No.])",-0.409414,0.303641,0.634193,0.303641,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
for i in admission_predict['Decision']:
    if i == 1:
        print(i)
    else:
        continue

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
admission_predict.columns.str.replace(' ', '_').lower()

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

----------------------------------------------------------- ITS ADDING TO 10 TO ALL ROWS EVEN IF UNI RATING BELOW 4----------------------

In [ ]:
for i in admission_predict['University Rating']:
    if i >= 4:
        admission_predict['adjusted_gre'] =  admission_predict['GRE Score'] + 10
    else:
        continue


In [ ]:
admission_predict['adjusted_gre']
bins = pd.cut(admission_predict['adjusted_gre'], 4)
bins.value_counts()

(325.0, 337.5]     140
(312.5, 325.0]     120
(337.5, 350.0]      70
(299.95, 312.5]     55
Name: adjusted_gre, dtype: int64